In [33]:
import json, difflib

import order, position, account
from account import Account

In [39]:
account.getAccountInfo()

{'id': '680e18e1-626a-411c-a2e2-cd4fe989451a',
 'account_number': 'PA38X9VR75LF',
 'status': 'ACTIVE',
 'currency': 'USD',
 'buying_power': '6000',
 'regt_buying_power': '6000',
 'daytrading_buying_power': '0',
 'cash': '3000',
 'portfolio_value': '3000',
 'pattern_day_trader': False,
 'trading_blocked': False,
 'transfers_blocked': False,
 'account_blocked': False,
 'created_at': '2021-07-20T08:06:50.257166Z',
 'trade_suspended_by_user': False,
 'multiplier': '2',
 'shorting_enabled': False,
 'equity': '3000',
 'last_equity': '3000',
 'long_market_value': '0',
 'short_market_value': '0',
 'initial_margin': '0',
 'maintenance_margin': '0',
 'last_maintenance_margin': '0',
 'sma': '0',
 'daytrade_count': 0}

In [7]:
myAccount = Account()
